In [1]:
import torch

print("Number of GPU: ", torch.cuda.device_count())
print("GPU Name: ", torch.cuda.get_device_name())


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Number of GPU:  1
GPU Name:  NVIDIA A100-SXM4-40GB
Using device: cuda


In [2]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from PIL import Image
import time
import os
import copy
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import time
import torch.nn.functional as F
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision import models
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from torchsummary import summary

In [3]:
import zipfile

zip_path = "/content/drive/MyDrive/fracture_detection/MURA-v1.1.zip"
extract_path = "/content/"
os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Unzipping completed!")

Unzipping completed!


In [4]:
class StudentCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),  # 224x224x3 -> 224x224x32
            nn.ReLU(),
            nn.MaxPool2d(2),  # -> 112x112x32
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # -> 56x56x64
            nn.Flatten(),
            nn.Linear(64 * 56 * 56, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        return self.model(x)


In [5]:
def distillation_loss(student_logits, teacher_logits, labels, temperature=4.0, alpha=0.5):
    # Hard label loss
    bce_loss = nn.BCEWithLogitsLoss()(student_logits, labels)

    # Soft label loss
    teacher_probs = torch.sigmoid(teacher_logits / temperature)
    student_probs = torch.sigmoid(student_logits / temperature)

    kl_loss = nn.KLDivLoss(reduction="batchmean")(
        torch.log(student_probs + 1e-8), teacher_probs
    )

    return alpha * bce_loss + (1 - alpha) * (temperature**2) * kl_loss


In [6]:
class CNNTransformerModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = torchvision.models.resnet50(pretrained=True)
        self.cnn.fc = nn.Identity()

        self.fc_proj = nn.Linear(2048, 768)

        encoder_layer = nn.TransformerEncoderLayer(d_model=768, nhead=8)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=1)

        self.classifier = nn.Linear(768, 1)

    def forward(self, x):
        x = self.cnn(x)
        x = self.fc_proj(x)
        x = x.unsqueeze(1)
        x = self.transformer(x)
        x = x.squeeze(1)
        x = self.classifier(x)
        return x


In [7]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


In [8]:
import pandas as pd

def load_paths_labels_from_csv(csv_file):
    df = pd.read_csv(csv_file, header=None)
    image_paths = []
    labels = []

    for i, row in df.iterrows():
        rel_path = row[0]
        label = 1 if "positive" in rel_path.lower() else 0
        full_path = os.path.join("/content", rel_path)

        if os.path.isfile(full_path):
            image_paths.append(full_path)
            labels.append(label)

    return image_paths, labels



In [9]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
from glob import glob
import torch
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import numpy as np


class MURADataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        if self.transform:
            image = self.transform(image)
        return image, label

def load_paths_labels(base_dir):
    image_paths, labels = [], []
    for phase in ["train", "valid"]:
        phase_dir = os.path.join(base_dir, phase)
        for body_part in os.listdir(phase_dir):
            body_part_dir = os.path.join(phase_dir, body_part)
            if not os.path.isdir(body_part_dir):
                continue
            for patient in os.listdir(body_part_dir):
                patient_dir = os.path.join(body_part_dir, patient)
                if not os.path.isdir(patient_dir):
                    continue
                for study in os.listdir(patient_dir):
                    study_dir = os.path.join(patient_dir, study)
                    if not os.path.isdir(study_dir):
                        continue
                    label = 1 if "positive" in study.lower() else 0
                    for img in glob(os.path.join(study_dir, "*.png")):
                        image_paths.append(img)
                        labels.append(label)
    return image_paths, labels


image_paths, labels = load_paths_labels("/content/MURA-v1.1")
labels = np.array(labels)

# 5-fold cross-validation setup
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kf.split(image_paths)):
    print(f"\n--- Fold {fold+1}/{k_folds} ---")

    train_paths = [image_paths[i] for i in train_idx]
    val_paths = [image_paths[i] for i in val_idx]
    train_labels = labels[train_idx]
    val_labels = labels[val_idx]

    train_dataset = MURADataset(train_paths, train_labels, transform=train_transform)
    val_dataset = MURADataset(val_paths, val_labels, transform=val_transform)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=2)





--- Fold 1/5 ---

--- Fold 2/5 ---

--- Fold 3/5 ---

--- Fold 4/5 ---

--- Fold 5/5 ---


In [14]:
teacher_model = CNNTransformerModel().to(device)
teacher_model.load_state_dict(torch.load("best_model.pth"))
teacher_model.eval()

student_model = StudentCNN().to(device)
optimizer = torch.optim.Adam(student_model.parameters(), lr=1e-4)
epochs = 10

for epoch in range(epochs):
    student_model.train()
    total_loss = 0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device).unsqueeze(1)

        with torch.no_grad():
            teacher_logits = teacher_model(imgs)

        student_logits = student_model(imgs)
        loss = distillation_loss(student_logits, teacher_logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}: Distillation Loss = {total_loss / len(train_loader):.4f}")


Epoch 1: Distillation Loss = -0.2796
Epoch 2: Distillation Loss = -0.3092
Epoch 3: Distillation Loss = -0.3341
Epoch 4: Distillation Loss = -0.3536
Epoch 5: Distillation Loss = -0.3703
Epoch 6: Distillation Loss = -0.3864
Epoch 7: Distillation Loss = -0.4040
Epoch 8: Distillation Loss = -0.4079
Epoch 9: Distillation Loss = -0.4220
Epoch 10: Distillation Loss = -0.4288


In [20]:
torch.save(student_model.state_dict(), "student_model.pth")  # Save model

from google.colab import files
files.download("student_model.pth")  # Download to your device


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

student_model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for imgs, labels in val_loader:
        imgs = imgs.to(device)
        labels = labels.to(device).unsqueeze(1)
        outputs = student_model(imgs)
        preds = (torch.sigmoid(outputs) > 0.5).float()
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

print("Student Model Evaluation:")
print(f"Accuracy  : {accuracy_score(y_true, y_pred):.4f}")
print(f"Precision : {precision_score(y_true, y_pred):.4f}")
print(f"Recall    : {recall_score(y_true, y_pred):.4f}")
print(f"F1 Score  : {f1_score(y_true, y_pred):.4f}")


Student Model Evaluation:
Accuracy  : 0.5738
Precision : 0.4333
Recall    : 0.1694
F1 Score  : 0.2436
